In [168]:
!pip install pandas requests

python(48864) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [169]:
pip install ntscraper

python(48869) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [170]:
!pip install firebase-admin


python(48872) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [171]:
pip install beautifulsoup4

python(48875) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Note: you may need to restart the kernel to use updated packages.


In [172]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
import os
import json
import firebase_admin
from firebase_admin import credentials, firestore, db
from bs4 import BeautifulSoup
from ntscraper import Nitter

In [173]:
service_account_path = os.getenv('FIREBASE_SERVICE_ACCOUNT_KEY')

In [174]:
if not firebase_admin._apps:
    cred = credentials.Certificate(service_account_path)
    firebase_admin.initialize_app(cred)

In [175]:
db = firestore.client()

In [176]:
scraper = Nitter()
scraper.working_instances = ['https://nitter.net']

Testing instances: 100%|██████████| 16/16 [00:20<00:00,  1.28s/it]


In [177]:
def getTweets(name, modes, num):
    total = scraper.get_tweets(name, mode=modes, number=num)
    print(total)
    
    final_tweets = []
    if 'tweets' in total:
        for tweet in total['tweets']:  # Adjust the key to match actual data
            data = [tweet['link'], tweet['text'], tweet['date']]
            final_tweets.append(data)
        data = pd.DataFrame(final_tweets, columns=['link', 'text', 'date'])
        return data
    else:
        print("Key 'tweets' not found in response")
        return None

In [189]:
data = getTweets('mpox', 'term', 25)
print(data)

04-Oct-24 09:33:37 - No instance specified, using random instance https://xcancel.com
04-Oct-24 09:33:40 - Empty page on https://xcancel.com


ValueError: Invalid instance

In [115]:
# Function to read existing JSON data (if any)
def read_existing_json(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            try:
                data = json.load(file)
                return data
            except json.JSONDecodeError:
                return {}
    else:
        return {}

# Function to append data to JSON without duplicates
def append_to_json(new_data, file_path):
    existing_data = read_existing_json(file_path)
    
    # Create a set of existing tweet links (to identify duplicates)
    existing_links = {tweet['link'] for tweet in existing_data.values()}
    
    # Append only new tweets (those not already in the existing JSON file)
    for i, tweet in enumerate(new_data):
        if tweet['link'] not in existing_links:
            existing_data[str(len(existing_data))] = tweet  # Use str(len) as the new index for appending
    
    # Write the updated data back to the JSON file
    with open(file_path, 'w') as file:
        json.dump(existing_data, file, indent=4)
    
    print("Data appended to JSON file successfully!")

In [ ]:
df = pd.DataFrame(data[['link', 'date', 'text']])
tweets_dict = df.to_dict(orient='records')
append_to_json(tweets_dict, 'tweets_filtered.json')
tweets_dict

In [123]:
def upload_to_firebase(data):
    collection_ref = db.collection('tweets')

    #records = data.to_dict(orient='records')
    
    for tweet in data:
        tweet_link = tweet["link"]
        existing_tweets = collection_ref.where('link', '==', tweet_link).stream()
        if not any(existing_tweets):
            doc_ref = collection_ref.document() 
            doc_ref.set(tweet)
            print(f"Uploaded tweet with ID {tweet_link}")
        else:
            print(f"Duplicate tweet with ID {tweet_link} skipped.")
    print("Data uploaded to Firebase successfully!")

In [ ]:
upload_to_firebase(tweets_dict)